Here we load our tools: Pandas for data, NumPy for math, and our two agents.

In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np

from strategy_agent import StrategyAgent
from risk_agent import RiskAgent


We’ll test on Apple’s enriched dataset again.

In [2]:
data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)


This computes CAGR, volatility, Sharpe, and drawdown — the KPIs RiskAgent needs.

In [3]:
def compute_metrics(df):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}


The StrategyAgent generates signals/returns for SMA, RSI, and Bollinger.

In [4]:
agent = StrategyAgent(data)

sma_res = agent.sma_crossover()
rsi_res = agent.rsi_strategy()
bb_res = agent.bollinger_breakout()


We compute metrics for each strategy separately so RiskAgent can judge them.

In [ ]:
strategies = {
    "SMA Crossover": sma_res,
    "RSI": rsi_res,
    "Bollinger": bb_res
}

metrics_results = {name: compute_metrics(df) for name, df in strategies.items()}
pd.DataFrame(metrics_results).T
